<a href="https://colab.research.google.com/github/DGuilherme/AI_Mixer/blob/main/Mixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install magenta
!pip install librosa
!apt-get install fluidsynth
!pip install midi2audio
!pip install matplotlib

In [ ]:
import numpy as np
import librosa
import note_seq
from note_seq import midi_io
from magenta.models.music_vae import TrainedModel
from magenta.models.music_vae import configs
from midi2audio import FluidSynth
import matplotlib.pyplot as plt
import IPython.display as ipd

In [ ]:
# Set up the MusicVAE model
config_name = 'hierdec-trio_16bar'
checkpoint = 'https://storage.googleapis.com/magentadata/models/music_vae/checkpoints/hierdec-trio_16bar.tar'
music_vae = TrainedModel(
    configs.CONFIG_MAP[config_name],
    batch_size=4,
    checkpoint_dir_or_path=checkpoint
)


In [ ]:
# Function to load MIDI files
def load_midi(midi_path):
    sequence = midi_io.midi_file_to_note_sequence(midi_path)
    return sequence

# Function to save MIDI files
def save_midi(sequence, output_path):
    midi_io.sequence_proto_to_midi_file(sequence, output_path)


In [ ]:
# Load MIDI files
midi_path1 = '/content/drive/My Drive/audio/track1.mid'
midi_path2 = '/content/drive/My Drive/audio/track2.mid'
sequence1 = load_midi(midi_path1)
sequence2 = load_midi(midi_path2)

In [ ]:
# Encode the MIDI sequences
encoded_1 = music_vae.encode([sequence1])
encoded_2 = music_vae.encode([sequence2])

# Mix the encoded tracks
ratio = 0.5  # You can adjust this ratio as needed
mixed_encoding = ratio * encoded_1 + (1 - ratio) * encoded_2

# Decode the mixed track
mixed_sequence = music_vae.decode(mixed_encoding)[0]

# Save the mixed sequence as a MIDI file
output_midi_path = '/content/drive/My Drive/audio/mixed_track.mid'
save_midi(mixed_sequence, output_midi_path)

# Convert MIDI to audio
fs = FluidSynth()
audio_output_path = '/content/drive/My Drive/audio/mixed_track.wav'
fs.midi_to_audio(output_midi_path, audio_output_path)

In [ ]:
# Load the audio file
y, sr = librosa.load(audio_output_path)

# Plot the waveform
plt.figure(figsize=(14, 5))
librosa.display.waveshow(y, sr=sr)
plt.title('Waveform of Mixed Track')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.show()

# Display the audio player
ipd.display(ipd.Audio(audio_output_path))

# Plot the spectrogram
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
plt.figure(figsize=(14, 5))
librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram of Mixed Track')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()

# Display the audio player again
ipd.display(ipd.Audio(audio_output_path))